In [7]:
import pandas as pd
import numpy as np
import kaggle
import re

In [ ]:
kaggle.api.authenticate()

# https://www.kaggle.com/oscarfry/tvtime-shows?select=tvtimeshows.csv
kaggle.api.dataset_download_files('oscarfry/tvtime-shows', path='./data/tvtimeshows', unzip = True)

# https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney 
kaggle.api.dataset_download_files('ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney', path='./data/streamings', unzip = True)

# https://www.kaggle.com/unanimad/emmy-awards
kaggle.api.dataset_download_files('unanimad/emmy-awards', path='./data/awards', unzip = True)]

# https://www.kaggle.com/unanimad/golden-globe-awards
kaggle.api.dataset_download_files('unanimad/golden-globe-awards', path='./data/awards' , unzip = True)

In [2]:
df_tvtime = pd.read_csv('data/tvtimeshows/tvtimeshows.csv')
# df_tvtime.head()

df_streamings = pd.read_csv('data/streamings/tv_shows.csv')
# df_streamings.head()

df_award_emmy = pd.read_csv('data/awards/the_emmy_awards.csv')
# df_award_emmy.head()

df_award_goldenglobe = pd.read_csv('data/awards/golden_globe_awards.csv')
# df_award_goldenglobe.head()

In [94]:
df_tvstatus = pd.read_csv('status/metacritic.csv')
df_tvstatus.head()

,name,season,status,tv_network,ano
0,20/20,SEASON 36,Renewed 5/13,ABC,2013
1,America's Funniest Home Videos,SEASON 24,Renewed 5/9,ABC,2013
2,The Assets,NaN,Canceled 1/10 after 2 eps aired,ABC,2013
3,The Astronaut Wives Club,Starts tbd,Delayed to 2015 for creative reasons,ABC,2013
4,The Bachelor,SEASON 18,Renewed 5/9,ABC,2013


In [95]:
def networks_more_popular(df_tt, networks): 
    column_names = list(df_tt.columns)
    df_most_popular = pd.DataFrame(columns = column_names)
    for i in networks:
        df = df_tt[df_tt['tv_network'].str.contains(i, flags=re.IGNORECASE, regex=True, na=False)]
        df_most_popular = df_most_popular.append(df,ignore_index=True)
        df_most_popular['tv_network'] = np.where(df_most_popular['tv_network'].str.contains(i, flags=re.IGNORECASE, regex=True, na=False),i,df_most_popular['tv_network'] )
    return df_most_popular

In [96]:
popular_networks = [ "ABC", "CBC", "NBC", "NETFLIX", "AMAZON", "DISNEY", "HULU", "HBO", "CRUNCHYROLL", "GLOBO", "APPLE", "FOX", "MTV",'FX','CBS','SYFY','THE CW','FREEF','TNT','IFC','STRZ','TVL','SHOWTIME','AMC','USA']
df_tvstatus_popular = networks_more_popular(df_tvstatus,popular_networks)

In [97]:
def standardize_status(df): 
    column_names = list(df.columns)
    df_clean = pd.DataFrame(columns = column_names)
    df_clean = df
    df_clean['status'] =  np.where(df_clean['status'].str.contains('CANCELED', flags=re.IGNORECASE, regex=True, na=False),'CANCELED',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('Renew' , flags=re.IGNORECASE, regex=True, na=False),'RENEWED',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('Delayed' , flags=re.IGNORECASE, regex=True, na=False),'DELAYED',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('Ending' , flags=re.IGNORECASE, regex=True, na=False),'END',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('ended' , flags=re.IGNORECASE, regex=True, na=False),'END',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('Final season' , flags=re.IGNORECASE, regex=True, na=False),'end',df_clean['status'] )
    df_clean['status'] =  np.where(df_clean['status'].str.contains('miniseries' , flags=re.IGNORECASE, regex=True, na=False),'MINISERIES',df_clean['status'] )
    nationality = ['Brazilian' , 'Spanish', 'German', 'Korean', 'Japanese', 'Swedish', 'French', 'Argentinian', 'Austalian', 'British', 'Belgian', 'Polish', 'Chinese', 'Italian', 'Mexican', 'Indian', 'Colombian', 'Thai', 'Taiwanese', 'Turkish', 'Norwegian', 'Lebanese', 'Israeli', 'Arabic', 'Dutch', 'Argentine']
    for i in nationality:
       df_clean['status'] =  np.where(df_clean['status'].str.contains(i , flags=re.IGNORECASE, regex=True, na=False),'NATIONALITY',df_clean['status'] ) 
    return(df_clean)


In [98]:
df_tvstatus_popular_cleaning = standardize_status(df_tvstatus_popular)

In [99]:
# df_tvstatus_popular_cleaning['tv_network'].unique()

In [100]:
df_tvstatus_popular_cleaning['status'].unique()

array(['RENEWED', 'CANCELED', 'DELAYED', nan,
       '1st season in ABC primetime',
       'Season cut to 10 episodes on 10/23', 'MINISERIES',
       'New version of classic game show', 'Postponed to summer',
       'Pulled from schedule on 3/29 after 5 episodes', 'END',
       'Pulled from schedule 1/4 (4 eps unaired)',
       'Banished to Saturdays effective 7/8',
       "Sneak preview aired in March;  returns later in '18",
       'Moved  to Saturdays 12/8; pulled 1/4',
       'Fate uncertain; not included in 2014-15 schedule',
       'Pulled from schedule on 10/18',
       'Pulled from schedule 2/5; 7 eps unaired',
       'Not on schedule; likely will never air', 'Variety show',
       'NATIONALITY', 'Two seasons (S3 & S4) ordered by Netflix',
       '2nd season already in production',
       'Continuation of 2000-07 WB/CW series',
       'Moves to Netflix from BBC America',
       'UK series formerly known as Scrotal Recall',
       'Original order is for 2 seasons', 'Canadian ser

In [101]:
df_tvstatus_popular_cleaning[df_tvstatus_popular_cleaning['status'].str.contains('Japanese series', flags=re.IGNORECASE, regex=True, na=False)]

,name,season,status,tv_network,ano
